## Data clean-up

In [47]:
import matplotlib.pyplot as plt
import pandas as pd
import os 

plt.ioff()

csfont = {'fontname':'AnjaliOldLipi', 'fontsize':30}
hfont = {'fontname':'AnjaliOldLipi', 'fontsize':30}

In [48]:
import wandb

api = wandb.Api()

def get_runs_filters(api, group="sfols", env_name = "Delivery-v0", timestep_col = "learning_timestep"):

    runs = api.runs("davidguillermo/sfcomp")
    
    summary_list, config_list, name_list = [], [], []

    df = pd.DataFrame(columns=["timestep"])
    
    for run in runs: 
        if run.group == group and run.config["env"]["gym_name"] == env_name:

            data = run.history(x_axis=timestep_col, keys=["learning/fsa_reward"])

            df.loc[:, "timestep"] = data[timestep_col]
            df.loc[:, run.name] = data["learning/fsa_reward"]
    
        # # .config contains the hyperparameters.
        # #  We remove special values that start with _.
        # config_list.append(
        #     {k: v for k,v in run.config.items()
        #       if not k.startswith('_')})
    
        # # .name is the human-readable name of the run.
        # name_list.append(run.name)

    return df

def drop_columns(df):
    todrop = []
    for col in df.columns:
        if '__MIN' in col or '__MAX' in col or (col.endswith("_step") and col !="global_step"):
            todrop.append(col) 
    todrop

    return df.drop(columns=todrop)


def cleandf(df):
    
    df = drop_columns(df)
    #rename 

    print(df.columns)

    renames = {}

    newdf = pd.DataFrame(columns=["step", "mean", "std"])

    newdf.loc[:, "mean"] = df.loc[ : , df.columns!='timestep'].mean(axis=1)
    newdf.loc[:, "std"] = df.loc[ : , df.columns!='timestep'].std(axis=1)
    newdf.loc[:, "step"] = df.timestep

    return newdf

In [28]:
delivery_sfols = cleandf(sfols_delivery)

sr, timemark = len(delivery_sfols), delivery_sfols.step.max()

mean, std = delivery_sfols.iloc[-1, [1,2]]

for i, r in enumerate(range(timemark+500, 160001, 500)):

    delivery_sfols.loc[sr + i + 1, :] = [r, mean, std]

Index(['timestep', 'stoic-music-567', 'misunderstood-cosmos-565',
       'dry-river-563', 'exalted-bee-562', 'vibrant-feather-560',
       'flowing-wildflower-559', 'deep-brook-557', 'vital-forest-556',
       'lyric-voice-554', 'tough-gorge-552', 'playful-rain-548',
       'fiery-music-597', 'fresh-bush-595', 'deft-oath-592',
       'laced-dragon-589', 'flowing-totem-587', 'royal-sound-584',
       'earnest-dew-582', 'dutiful-shape-579', 'still-lake-577',
       'eager-meadow-574', 'balmy-sun-571', 'chocolate-spaceship-570',
       'ancient-morning-569', 'swift-flower-567', 'sunny-violet-564',
       'cerulean-water-561', 'devoted-sunset-559', 'wild-jazz-556',
       'radiant-wood-554'],
      dtype='object')


In [29]:
delivery_sfols

,step,mean,std
0,500,-1001.000000,1.286535
1,1000,-1001.300000,1.878187
2,1500,-1000.600000,0.674665
3,2000,-1001.600000,1.773366
4,2500,-1000.700000,1.290549
...,...,...,...
316,158000,-44.666667,12.576204
317,158500,-44.666667,12.576204
318,159000,-44.666667,12.576204
319,159500,-44.666667,12.576204


### Delivery

In [40]:
# sfols_delivery = get_runs_filters(api)
# delivery_sfols = cleandf(sfols_delivery)

# lof_delivery = get_runs_filters(api, group="lof", env_name = "Delivery-v0", timestep_col = "learning_timestep")
# delivery_lof = cleandf(lof_delivery)

Index(['timestep', 'stoic-music-567', 'misunderstood-cosmos-565',
       'dry-river-563', 'exalted-bee-562', 'vibrant-feather-560',
       'flowing-wildflower-559', 'deep-brook-557', 'vital-forest-556',
       'lyric-voice-554', 'tough-gorge-552', 'playful-rain-548',
       'fiery-music-597', 'fresh-bush-595', 'deft-oath-592',
       'laced-dragon-589', 'flowing-totem-587', 'royal-sound-584',
       'earnest-dew-582', 'dutiful-shape-579', 'still-lake-577',
       'eager-meadow-574', 'balmy-sun-571', 'chocolate-spaceship-570',
       'ancient-morning-569', 'swift-flower-567', 'sunny-violet-564',
       'cerulean-water-561', 'devoted-sunset-559', 'wild-jazz-556',
       'radiant-wood-554'],
      dtype='object')


In [41]:
delivery_sfols

mean, std = delivery_sfols.iloc[-1, [-2, -1]]

ts = len(delivery_sfols)

iter = range(delivery_sfols.step.max()+500, 160001, 500)
for i, t in enumerate(iter):
    delivery_sfols.loc[ts+i+1, :] = [t, mean, std]

In [42]:
delivery_lof

,step,mean,std
0,500,-945.896552,206.604582
1,1000,-946.103448,206.668013
2,1500,-752.068966,376.89492
3,2000,-366.068966,330.903086
4,2500,-227.620690,148.741966
...,...,...,...
315,158000,-48.206897,10.004925
316,158500,-48.206897,10.004925
317,159000,-48.206897,10.004925
318,159500,-48.206897,10.004925


In [43]:
delivery_sfols.to_csv("SFOLS-Delivery.csv", index=False)
delivery_lof.to_csv("LOF-Delivery.csv", index=False)

### Office

In [55]:
# lof_office = get_runs_filters(api, group="lof", env_name = "Office-v0", timestep_col = "learning_timestep")
# office_lof = cleandf(lof_office)

sfols_office = get_runs_filters(api, group="sfols", env_name = "Office-v0", timestep_col = "learning_timestep")
office_sfols = cleandf(sfols_office)

Index(['timestep', 'breezy-water-702', 'snowy-energy-701',
       'dazzling-valley-700', 'flowing-yogurt-699', 'resilient-haze-698',
       'volcanic-brook-697', 'clean-planet-696', 'curious-sea-695',
       'radiant-shadow-694', 'fiery-frog-693', 'valiant-haze-692',
       'winter-dawn-691', 'fast-waterfall-690', 'dulcet-cherry-689',
       'twilight-bush-688', 'cerulean-galaxy-687', 'upbeat-terrain-686',
       'true-surf-685', 'graceful-silence-684', 'grateful-haze-683',
       'ancient-universe-682', 'denim-cosmos-681', 'good-firebrand-680',
       'smart-breeze-679', 'likely-plant-678', 'leafy-elevator-677',
       'crimson-energy-676', 'dazzling-lake-675', 'polished-dragon-669',
       'jolly-sea-664'],
      dtype='object')


In [50]:
mean, std = office_sfols.iloc[-1, [-2, -1]]

ts = len(office_sfols)

iter = range(office_sfols.step.max()+500, 160001, 500)
for i, t in enumerate(iter):
    office_sfols.loc[ts+i+1, :] = [t, mean, std]

TypeError: 'float' object cannot be interpreted as an integer

In [57]:
sfols_office

,timestep,breezy-water-702,snowy-energy-701,dazzling-valley-700,flowing-yogurt-699,resilient-haze-698,volcanic-brook-697,clean-planet-696,curious-sea-695,radiant-shadow-694,...,ancient-universe-682,denim-cosmos-681,good-firebrand-680,smart-breeze-679,likely-plant-678,leafy-elevator-677,crimson-energy-676,dazzling-lake-675,polished-dragon-669,jolly-sea-664
0,500.0,-155,-200.0,-200.0,-200.0,-200.0,-200.0,-179.0,-200.0,-200.0,...,-200.0,-200.0,-200.0,-200.0,-179.0,-162,-200.0,-200.0,-200.0,-200.0
1,1000.0,-200,-200.0,-200.0,-200.0,-176.0,-200.0,-200.0,-200.0,-200.0,...,-200.0,-200.0,-176.0,-200.0,-200.0,-200,-200.0,-200.0,-200.0,-200.0
2,1500.0,-200,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,...,-200.0,-130.0,-200.0,-200.0,-200.0,-200,-200.0,-200.0,-200.0,-200.0
3,2000.0,-200,-200.0,-39.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,...,-27.0,-200.0,-200.0,-65.0,-200.0,-200,-200.0,-200.0,-200.0,-200.0
4,2500.0,-200,-200.0,-38.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,...,-27.0,-200.0,-200.0,-200.0,-200.0,-200,-200.0,-200.0,-200.0,-200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,NaN,-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-22,NaN,NaN,NaN,NaN
304,NaN,-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-22,NaN,NaN,NaN,NaN
305,NaN,-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-22,NaN,NaN,NaN,NaN
306,NaN,-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-22,NaN,NaN,NaN,NaN


In [36]:
office_sfols.to_csv("SFOLS-Office.csv", index=False)
office_lof.to_csv("LOF-Office.csv", index=False)